
# Ejemplo RAG Simple

In [130]:
import numpy as np
import pandas as pd
from docling.document_converter import DocumentConverter
from dotenv import load_dotenv

from IPython.display import Markdown, display
from gensim.models import Word2Vec
from langchain_text_splitters import RecursiveCharacterTextSplitter, MarkdownHeaderTextSplitter

import os
import google.generativeai as genai


load_dotenv("../.env")

True

## Extraccion de Texto - OCR

In [4]:
source = "../docs/Proyecto Final - Curso MLE2.pdf"  # PDF path or URL
converter = DocumentConverter()
result = converter.convert(source)
print(result.document.export_to_markdown())

2025-10-21 19:40:37,888 - INFO - detected formats: [<InputFormat.PDF: 'pdf'>]


2025-10-21 19:40:38,093 - INFO - Going to convert document batch...
2025-10-21 19:40:38,096 - INFO - Initializing pipeline for StandardPdfPipeline with options hash 4f2edc0f7d9bb60b38ebfecf9a2609f5
2025-10-21 19:40:38,145 - INFO - Loading plugin 'docling_defaults'
2025-10-21 19:40:38,156 - INFO - Registered picture descriptions: ['vlm', 'api']
2025-10-21 19:40:38,213 - INFO - Loading plugin 'docling_defaults'
2025-10-21 19:40:38,232 - INFO - Registered ocr engines: ['auto', 'easyocr', 'ocrmac', 'rapidocr', 'tesserocr', 'tesseract']
2025-10-21 19:40:38,236 - INFO - rapidocr cannot be used because onnxruntime is not installed.
2025-10-21 19:40:38,241 - INFO - easyocr cannot be used because it is not installed.
2025-10-21 19:40:39,491 - INFO - Accelerator device: 'cpu'
[INFO] 2025-10-21 19:40:39,546 [RapidOCR] base.py:22: Using engine_name: torch
[INFO] 2025-10-21 19:40:39,689 [RapidOCR] download_file.py:60: File exists and is valid: /home/keras/wordspaces/ml2_clases/.venv/lib/python3.12/

<!-- image -->

Science

## Proyecto Curso II -Especialización Machine Learning Engineering

El proyecto del curso II de la especialización de Machine Learning Engineering tiene como objetivo enfrentar al estudiante a:

- -Conocimiento  teórico  de  algoritmos  populares  y  avanzados  de  Machine Learning en la industria
- -Administración de ciclo de vida de modelos de Machine Learning
1.  README.md
- a.  Problema de ML
- b.  Diagrama de flujo del proyecto
- c. Descripción del dataset con su respectivo diccionario de datos
- d.  Model Card https://www.kaggle.com/code/var0101/model-cards
- e.  Resultados con Métricas de evaluación offline y online
- f. Conclusiones
2.  Estructura del repositorio de código con:
- a.  Carpeta de notebooks
- i.  Notebook preprocesamiento de datos
- ii.  Notebook de Machine Learning
- b.  Carpeta de datos (.csv, .txt, .parquet)
- c.  Módulo de código reusable
- d.  Scripts de ejecución (Preprocesamiento, entrenamiento y predicción)
3.  Link con evidencia d

## Chunking

#### Metodo 1 de Chunking -> Chunking de Tamaño Fijo

In [66]:
def fixed_window_splitter(text: str, chunk_size: int = 500, chunk_overlap: int = 0) -> list[str]:
    """Corta el texto en pedazos de tamaño fijo."""
    if chunk_overlap >= chunk_size:
        raise ValueError("El solapamiento (overlap) no puede ser mayor o igual al tamaño del chunk.")
        
    chunks = []
    start = 0
    while start < len(text):
        end = start + chunk_size
        chunks.append(text[start:end])
        start += chunk_size - chunk_overlap
    return chunks

In [ ]:
texts= fixed_window_splitter(result.document.export_to_markdown(), chunk_size=500, chunk_overlap=50)
print("\n--- Método 1: Tamaño Fijo ---", len(texts))

for i, chunk in enumerate(texts):
    print(f"Chunk {i+1}: '{chunk}'")

#### Metodo 2 de Chunking -> Chunking Recursivo por Caracteres

In [113]:
def recursive_character_splitter(text: str, chunk_size: int = 500, chunk_overlap: int = 50) -> list[str]:
    """Corta el texto usando un enfoque recursivo basado en separadores."""
    text_splitter_recursive = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap
    )

    return text_splitter_recursive.split_text(text)

In [114]:
texts = recursive_character_splitter(result.document.export_to_markdown(), chunk_size=500, chunk_overlap=50)

print("\n--- Método 2: Recursivo por Caracteres ---", len(texts))

for i, chunk in enumerate(texts):
    print(f"Chunk {i+1}: '{chunk}'")
    print("-"*20)


--- Método 2: Recursivo por Caracteres --- 21
Chunk 1: '<!-- image -->

Science

## Proyecto Curso II -Especialización Machine Learning Engineering

El proyecto del curso II de la especialización de Machine Learning Engineering tiene como objetivo enfrentar al estudiante a:'
--------------------
Chunk 2: '- -Conocimiento  teórico  de  algoritmos  populares  y  avanzados  de  Machine Learning en la industria
- -Administración de ciclo de vida de modelos de Machine Learning
1.  README.md
- a.  Problema de ML
- b.  Diagrama de flujo del proyecto
- c. Descripción del dataset con su respectivo diccionario de datos
- d.  Model Card https://www.kaggle.com/code/var0101/model-cards
- e.  Resultados con Métricas de evaluación offline y online
- f. Conclusiones'
--------------------
Chunk 3: '- f. Conclusiones
2.  Estructura del repositorio de código con:
- a.  Carpeta de notebooks
- i.  Notebook preprocesamiento de datos
- ii.  Notebook de Machine Learning
- b.  Carpeta de datos (.csv, .txt, .p

#### Metodo 3 de Chunking -> Chunking Consciente del Contenido (Markdown)

In [89]:
def markdown_header_splitter(text: str) -> list[str]:
    """Corta el texto usando encabezados de Markdown como separadores."""
    # 1. Define por qué encabezados quieres dividir el texto
    headers_to_split_on = [
        ("#", "Header 1"),
        ("##", "Header 2"),
        ("###", "Header 3"),
    ]

    # 2. Configura el divisor de Markdown
    markdown_splitter = MarkdownHeaderTextSplitter(headers_to_split_on=headers_to_split_on)

    # Esto devuelve la lista de objetos Document
    chunks_with_metadata = markdown_splitter.split_text(text)

    # Extraemos solo el .page_content de cada chunk
    text_only_chunks = [chunk.page_content for chunk in chunks_with_metadata]

    return text_only_chunks

In [ ]:
texts = markdown_header_splitter(result.document.export_to_markdown())

print("\n--- Método 3: Consciente del Contenido (Markdown) ---", len(texts))

for i, chunk in enumerate(texts):
    print(f"Chunk {i+1}: '{chunk}'")
    print("-"*20)

## Pregunta RAG: QUESTION = '¿Cuáles son los criterios de evaluación?'

In [81]:
QUESTION = '¿Cuáles son los criterios de evaluación?'

### Word2Vec Embeddings

In [17]:
import nltk
from nltk.tokenize import sent_tokenize
from gensim.utils import simple_preprocess
from nltk.corpus import stopwords

# Descargar recursos de NLTK si es la primera vez
nltk.download('punkt')  #Tokenizador
nltk.download('stopwords') #Stopwords
nltk.download('punkt_tab') #Tokenizador para tablas

# Lista de stopwords en español
stop_words = set(stopwords.words('spanish'))

[nltk_data] Downloading package punkt to /home/keras/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/keras/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to /home/keras/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


In [96]:
def preprocess_text_spanish(text):
    # Paso 1: dividir en oraciones
    sentences = sent_tokenize(text, language='spanish')
    
    # Paso 2: tokenizar oraciones y limpiar
    tokenized_sentences = []
    for sentence in sentences:
        tokens = simple_preprocess(sentence, deacc=True)  # deacc=True quita tildes y convierte a minúsculas
        # La divide en una lista de palabras individuales (tokens)
        # Elimina la puntuación y convierte a minúsculas

        filtered = [word for word in tokens if word not in stop_words and len(word) > 2] # Elimina stopwords y palabras muy cortas
        tokenized_sentences.append(filtered)
        
    return tokenized_sentences

In [97]:
tokens = preprocess_text_spanish(result.document.export_to_markdown())
print(tokens[:1])  # Mostrar las primeras 3 oraciones tokenizadas

[['image', 'science', 'proyecto', 'curso', 'especializacion', 'machine', 'learning', 'engineering', 'proyecto', 'curso', 'especializacion', 'machine', 'learning', 'engineering', 'objetivo', 'enfrentar', 'estudiante', 'conocimiento', 'teorico', 'algoritmos', 'populares', 'avanzados', 'machine', 'learning', 'industria', 'administracion', 'ciclo', 'vida', 'modelos', 'machine', 'learning']]


In [115]:
modelo = Word2Vec(
    tokens, 
    vector_size=60, # Dimensionalidad del espacio vectorial
    window=5, #Es el tamaño de la ventana de contexto. Para aprender el significado de una palabra, el modelo mirará las 5 palabras que la preceden y las 5 que la siguen
    min_count=1, # Mínimo número de ocurrencias para considerar una palabra
    sg=1 # Usar Skip-gram (sg=1) en lugar de CBOW (sg=0
)

2025-10-22 14:39:51,979 - INFO - collecting all words and their counts
2025-10-22 14:39:51,981 - INFO - PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2025-10-22 14:39:51,982 - INFO - collected 280 word types from a corpus of 473 raw words and 30 sentences
2025-10-22 14:39:51,983 - INFO - Creating a fresh vocabulary
2025-10-22 14:39:51,985 - INFO - Word2Vec lifecycle event {'msg': 'effective_min_count=1 retains 280 unique words (100.00% of original 280, drops 0)', 'datetime': '2025-10-22T14:39:51.985158', 'gensim': '4.4.0', 'python': '3.12.10 (main, Sep 30 2025, 03:55:04) [GCC 11.4.0]', 'platform': 'Linux-6.6.87.2-microsoft-standard-WSL2-x86_64-with-glibc2.35', 'event': 'prepare_vocab'}
2025-10-22 14:39:51,986 - INFO - Word2Vec lifecycle event {'msg': 'effective_min_count=1 leaves 473 word corpus (100.00% of original 473, drops 0)', 'datetime': '2025-10-22T14:39:51.986105', 'gensim': '4.4.0', 'python': '3.12.10 (main, Sep 30 2025, 03:55:04) [GCC 11.4.0]', 'platform':

In [99]:
modelo.wv["image"]  # Vector de la palabra "image"

array([-0.01344557, -0.00131021,  0.00295761,  0.01248962, -0.0039169 ,
       -0.00389292,  0.0170833 ,  0.00108022, -0.00367756,  0.01471856,
        0.00575105, -0.00885321,  0.01134294,  0.0071739 , -0.01139473,
       -0.00119772,  0.002078  ,  0.00909631, -0.00383826, -0.00534405,
        0.00314703,  0.00078638,  0.00241346, -0.00394885, -0.00993309,
        0.01198021,  0.01226117,  0.01406053, -0.01477605,  0.00430212,
       -0.00594526,  0.01488244,  0.00541708,  0.00799153,  0.00541288,
        0.01223443, -0.0103472 ,  0.01220585, -0.00533314, -0.00715146,
       -0.00370761,  0.01657126, -0.01154723, -0.00371517,  0.01151392,
       -0.00011743, -0.01071196, -0.01058971,  0.01458341,  0.01167425,
        0.00826388, -0.00568103, -0.01513977,  0.0073448 ,  0.01254901,
       -0.00877339, -0.01302378, -0.00525597, -0.0072409 , -0.004888  ],
      dtype=float32)

In [116]:
new_tokens = simple_preprocess(QUESTION, deacc=True)
filtered_tokens = [w for w in new_tokens if w not in stop_words and len(w) > 2]

def sentence_vector(model, sentence_tokens):
    vectors = [model.wv[word] for word in sentence_tokens if word in model.wv] # Obtener vectores de palabras conocidas
    if vectors:
        return np.mean(vectors, axis=0)
    else:
        return np.zeros(model.vector_size)

# Encode
vec1 = sentence_vector(modelo, filtered_tokens)

print("Vector de la pregunta:", vec1)

Vector de la pregunta: [ 0.0021417   0.01739228  0.01566267  0.01314747 -0.01113332  0.01685121
  0.0112926   0.01621915 -0.00301806  0.00570141  0.01263748 -0.00638959
  0.01184658  0.00390362  0.01566871 -0.00571403 -0.00468942  0.00976107
 -0.0162175  -0.0129274  -0.01059293 -0.00200072 -0.0036456   0.01198017
  0.01575762 -0.00148535  0.00170695  0.00372894 -0.00374415 -0.01374721
  0.01275316 -0.00803406  0.00828245  0.01545073  0.00651095  0.00652246
  0.00598794 -0.00561134  0.00502371  0.00062585  0.00931662  0.00946036
  0.00271805  0.00964665 -0.01507252  0.01099488  0.0152591  -0.00699815
  0.0021947  -0.00750969  0.00197478  0.00458855  0.01390576  0.0018398
  0.00507629  0.00291774  0.0141964  -0.00994971 -0.00955671 -0.00731092]


In [117]:
similarities = {}
for i in range(len(texts)):
    print(f"Texto {i}")
    text_tokens = [w for w in simple_preprocess(texts[i], deacc=True) if w not in stop_words and len(w) > 2]
    vec2 = sentence_vector(modelo, text_tokens)

    # Compute cosine similarity
    similarity = np.dot(vec1, vec2) / (np.linalg.norm(vec1) * np.linalg.norm(vec2))
    similarities[i] = similarity

    print(f"Similitud: {similarity.item():.4f}")

best_candidates =  [i[0] for i in sorted(similarities.items(), key=lambda item: item[1], reverse=True)[:5]]
word2_vec_context = [texts[i] for i in best_candidates]

Texto 0
Similitud: 0.1537
Texto 1
Similitud: 0.2761
Texto 2
Similitud: 0.2377
Texto 3
Similitud: 0.2420
Texto 4
Similitud: 0.0918
Texto 5
Similitud: 0.2086
Texto 6
Similitud: -0.1296
Texto 7
Similitud: 0.2112
Texto 8
Similitud: 0.3411
Texto 9
Similitud: 0.1284
Texto 10
Similitud: 0.7303
Texto 11
Similitud: 0.5464
Texto 12
Similitud: nan
Texto 13
Similitud: 0.0892
Texto 14
Similitud: 0.1493
Texto 15
Similitud: -0.1867
Texto 16
Similitud: 0.0973
Texto 17
Similitud: 0.1919
Texto 18
Similitud: 0.1655
Texto 19
Similitud: 0.1821
Texto 20
Similitud: 0.1829


/tmp/ipykernel_1087/3180159885.py:8: RuntimeWarning: invalid value encountered in scalar divide
  similarity = np.dot(vec1, vec2) / (np.linalg.norm(vec1) * np.linalg.norm(vec2))


In [118]:
for sentence in word2_vec_context:
    print(f"- {sentence}")

- <!-- image -->

## Evaluación
- | Componente                     | Tipo        | Descripción                                                                                                                                                                                                                                                               | Porcentaje en la evaluación                                  |
- - -Revisar la guía básica de Markdown para preparar el README.md con mejor estructura. https://markdown.es/
- -Agregar  excepciones  adicionales  para  archivos  que  estemos  manejando. https://docs.github.com/es/get-started/git-basics/ignoring-files
- -Usar. gitkeep en carpetas provisionales del proyecto (data, tmp, etc).
- - -Conocimiento  teórico  de  algoritmos  populares  y  avanzados  de  Machine Learning en la industria
- -Administración de ciclo de vida de modelos de Machine Learning
1.  README.md
- a.  Problema de ML
- b.  Diagrama de flujo del proyecto
- c. Descripci

### Sentence Transformers Embeddings

In [153]:
from sentence_transformers import SentenceTransformer, util

model = SentenceTransformer('hiiamsid/sentence_similarity_spanish_es')

2025-10-22 17:47:56,126 - INFO - Use pytorch device_name: cpu
2025-10-22 17:47:56,134 - INFO - Load pretrained SentenceTransformer: hiiamsid/sentence_similarity_spanish_es


In [154]:
embedding1 = model.encode(QUESTION, convert_to_tensor=True)

similarities = {}
for i in range(len(texts)):
    print(f"Texto {i}")
    embedding2 = model.encode(texts[i], convert_to_tensor=True)

    # Compute cosine similarity
    similarity = util.pytorch_cos_sim(embedding1, embedding2)
    similarities[i] = similarity.item()

    print(f"Similitud: {similarity.item():.4f}")

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches: 100%|██████████| 1/1 [00:01<00:00,  1.06s/it]


Texto 0


Batches: 100%|██████████| 1/1 [00:00<00:00,  5.99it/s]


Similitud: 0.1939
Texto 1


...

Similitud: 0.1330
Texto 19


Batches: 100%|██████████| 1/1 [00:00<00:00,  5.75it/s]


Similitud: 0.1260
Texto 20


Batches: 100%|██████████| 1/1 [00:00<00:00, 10.11it/s]

Similitud: 0.1418


In [126]:
embedding1

tensor([-3.3596e-01,  3.5151e-01,  1.5943e-01, -1.0919e+00,  5.6617e-02,
         6.5108e-01,  3.5689e-01, -3.2635e-01, -7.7893e-01,  7.8155e-01,
        -3.4961e-01, -8.8840e-02,  1.1152e-01, -4.7087e-01, -6.0106e-01,
        -1.1122e+00,  6.8781e-01, -8.2182e-02, -4.2655e-01, -7.6112e-01,
        -7.1291e-01, -8.9418e-01,  1.3327e+00,  1.0333e+00, -6.0296e-02,
        -2.5595e-01,  5.8809e-01,  4.9520e-01, -7.5380e-02,  1.1351e-01,
         6.8744e-01, -9.0994e-01,  8.9935e-01, -8.9519e-01,  4.0033e-01,
         5.8460e-01,  6.8219e-01, -5.7879e-01, -3.9665e-01,  8.0582e-01,
        -4.0264e-01,  4.9516e-01, -6.2085e-02,  2.1411e+00, -3.9417e-01,
         1.0503e+00,  5.1220e-01, -5.4658e-02,  4.2341e-01,  3.6144e-01,
        -1.1417e+00, -2.4956e-01,  3.6718e-01,  1.7407e-01, -7.2121e-02,
        -8.2703e-01,  8.1673e-01,  1.1948e+00,  3.6988e-02,  6.6886e-01,
        -3.0125e-01,  1.3759e+00, -9.2062e-02,  4.2017e-01,  3.7630e-01,
        -4.7959e-01, -3.7643e-01,  2.0472e-01, -6.7

In [155]:
best_candidates =  [i[0] for i in sorted(similarities.items(), key=lambda item: item[1], reverse=True)[:5]]
hf_context = [texts[i] for i in best_candidates]
hf_context

['| Modelo de Machine Learning     | Obligatorio | El estudiante documenta y desarrolla un modelo de machine learning alineado con sus hipótesis, además de evaluarlo con métricas que tengan sentido para el problema que previamente escogió y usando mlflow como herramienta de administración de modelos y experimentos. | 50%                                                          |',
 '| Reto ML 1                      | Opcional    | El estudiante demuestra dominio de los algoritmos vistos en clase realizando experimentos con más de uno y obteniendo resultados excepcionales (estos dependerán e tipo de problema que se busque resolver)                                                               | 10% [Acumulable para todos los cursos de la especialización] |',
 '<!-- image -->\n\n## Evaluación',
 '| Buenas prácticas de desarrollo | Obligatorio | El estudiante sigue el proceso estándar de desarrollo basado en control de versiones, usando commits, pull requests y releases en su proyecto.   

### Embeddings with Gemini

In [136]:
genai.configure(api_key=os.getenv("GOOGLE_API_KEY"))

In [141]:
# 1. Crea el embedding para la pregunta (esto sigue siendo una sola llamada)
response_q = genai.embed_content(
    model="gemini-embedding-001",
    content=QUESTION
)

vec1 = response_q['embedding']

response_chunks = genai.embed_content(
    model="gemini-embedding-001",
    content=texts  # <-- ¡Le pasas la lista completa de textos aquí!
)

all_vec2 = response_chunks['embedding']

# 3. Calcula la similitud del coseno (ahora es más fácil con NumPy)
#    No necesitas un bucle. Puedes hacerlo todo con operaciones de vectores.
similarities = np.dot(all_vec2, vec1) / (np.linalg.norm(all_vec2, axis=1) * np.linalg.norm(vec1))


# 4. Encuentra los mejores candidatos (más directo)
#    np.argsort() te da los índices de los valores ordenados.
best_candidates_indices = np.argsort(similarities)[::-1][:5] # Ordena de mayor a menor y toma los 5 primeros

gemini_context = [texts[i] for i in best_candidates_indices]

print("\n--- Contexto más relevante encontrado por Gemini ---")
print(gemini_context)


--- Contexto más relevante encontrado por Gemini ---
['<!-- image -->\n\n## Evaluación', '| Componente                     | Tipo        | Descripción                                                                                                                                                                                                                                                               | Porcentaje en la evaluación                                  |', '| Repositorio de Github          | Obligatorio | El estudiante presenta su proyecto de acuerdo con la estructura definida por el profesor.                                                                                                                                                                                 | 25%                                                          |', '| Buenas prácticas de desarrollo | Obligatorio | El estudiante sigue el proceso estándar de desarrollo basado en control de versiones, usando c

In [159]:
response_q = genai.embed_content(
    model="gemini-embedding-001",
    content=QUESTION,
    output_dimensionality= 768 
)
vec1 = response_q['embedding'] # El vector está en la clave 'embedding'

# 2. Bucle para comparar con cada chunk de texto
similarities = {}
for i in range(len(texts)):
    
    # Crea el embedding para el chunk actual
    response_chunk = genai.embed_content(
        model="gemini-embedding-001",
        content=texts[i],
        output_dimensionality= 768
        
    )
    vec2 = response_chunk['embedding']
    
    # 3. El cálculo de similitud del coseno NO CAMBIA
    similarity = np.dot(vec1, vec2) / (np.linalg.norm(vec1) * np.linalg.norm(vec2))
    similarities[i] = similarity

    print(f"Similitud: {i} {similarity:.4f}")

# 4. La selección de los mejores candidatos TAMPOCO CAMBIA
best_candidates = [i[0] for i in sorted(similarities.items(), key=lambda item: item[1], reverse=True)[:5]]
gemini_context = [texts[i] for i in best_candidates]

Similitud: 0.5765
Similitud: 0.5566
Similitud: 0.6088
Similitud: 0.6113
Similitud: 0.6097
Similitud: 0.5580
Similitud: 0.6208
Similitud: 0.6064
Similitud: 0.5494
Similitud: 0.5204
Similitud: 0.7413
Similitud: 0.7399
Similitud: 0.5255
Similitud: 0.6465
Similitud: 0.6262
Similitud: 0.6245
Similitud: 0.6061
Similitud: 0.6130
Similitud: 0.5330
Similitud: 0.5934
Similitud: 0.5309


In [160]:
for sentence in gemini_context:
    print("Inicio de chunk")
    print(f"- {sentence}", sep="\n")
    print("-"*20)

Inicio de chunk
- <!-- image -->

## Evaluación
--------------------
Inicio de chunk
- | Componente                     | Tipo        | Descripción                                                                                                                                                                                                                                                               | Porcentaje en la evaluación                                  |
--------------------
Inicio de chunk
- | Repositorio de Github          | Obligatorio | El estudiante presenta su proyecto de acuerdo con la estructura definida por el profesor.                                                                                                                                                                                 | 25%                                                          |
--------------------
Inicio de chunk
- | Modelo de Machine Learning     | Obligatorio | El estudiante documenta y 

## Construcción Final RAG

### GEMINI

In [ ]:
# uv add google-generativeai
import os
import google.generativeai as genai

genai.configure(api_key=os.getenv("GOOGLE_API_KEY"))


PROMPT = f"""Responde a la siguiente pregunta dado un CONTEXTO que contiene información para responder esa pregunta:

PREGUNTA
-------------------
{QUESTION}

CONTEXTO:
{''.join(gemini_context)}
---------------------

Crea el resultado final como un texto plano con formato markdown que pueda ser renderizado (evita incluir ```markdown como apertura)"""

model = genai.GenerativeModel("gemini-2.5-flash") # gemini-2.5-pro

# 2. Envía el prompt al modelo para generar la respuesta
response = model.generate_content(PROMPT)

In [167]:
display(
    Markdown(response.text)  
)

Los criterios de evaluación son los siguientes:

*   **Repositorio de Github (25%):** El estudiante presenta su proyecto de acuerdo con la estructura definida por el profesor.
*   **Modelo de Machine Learning (50%):** El estudiante documenta y desarrolla un modelo de machine learning alineado con sus hipótesis, lo evalúa con métricas pertinentes al problema y usa mlflow como herramienta de administración.
*   **Buenas prácticas de desarrollo (15%):** El estudiante sigue el proceso estándar de desarrollo basado en control de versiones, usando commits, pull requests y releases en su proyecto.

In [162]:
display(
    Markdown(response.text)
)

Los criterios de evaluación son los siguientes:

*   **Modelo de Machine Learning (Obligatorio):** El estudiante documenta y desarrolla un modelo de machine learning alineado con sus hipótesis, además de evaluarlo con métricas que tengan sentido para el problema que previamente escogió y usando mlflow como herramienta de administración de modelos y experimentos. (50%)
*   **Reto ML 1 (Opcional):** El estudiante demuestra dominio de los algoritmos vistos en clase realizando experimentos con más de uno y obteniendo resultados excepcionales (estos dependerán del tipo de problema que se busque resolver). (10% [Acumulable para todos los cursos de la especialización])
*   **Buenas prácticas de desarrollo (Obligatorio):** El estudiante sigue el proceso estándar de desarrollo basado en control de versiones, usando commits, pull requests y releases en su proyecto. (15%)
*   **Repositorio de Github (Obligatorio):** El estudiante presenta su proyecto de acuerdo con la estructura definida por el profesor. (25%)